In [5]:
import requests
from pymongo import MongoClient
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget
import os
import re
from dotenv import load_dotenv
from pandas.io.json import json_normalize
from functions import *

In [7]:
#para poder traer el database
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll
db, coll = connectCollection('companies','companies')
pipeline = [{ "$unwind": "$offices"},]
results = list(coll.aggregate(pipeline))

In [8]:
#creo el DataFrame
df = pd.DataFrame(results)
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,[],[],[],[],11.0,1.0,9.0,15.0,,None
3,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN
4,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN


In [9]:
#amplio en columnas nuevas los resultados de offices y borro la columna offices
df = pd.concat([df.drop(['offices'], axis=1), df['offices'].apply(pd.Series)], axis=1)

In [10]:
display(df.head(5))


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,NaN,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,NaN,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,None,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,NaN,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
4,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,NaN,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431


In [11]:
#estas son las columnas que han quedado
df.columns.unique()

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list',
       'alias_list', 'email_address', 'phone_number', 'description',
       'created_at', 'updated_at', 'overview', 'image', 'products',
       'relationships', 'competitions', 'providerships', 'total_money_raised',
       'funding_rounds', 'investments', 'acquisition', 'acquisitions',
       'milestones', 'video_embeds', 'screenshots', 'external_links',
       'partners', 'founded_month', 'founded_day', 'deadpooled_month',
       'deadpooled_day', 'deadpooled_url', 'ipo', 'address1', 'address2',
       'zip_code', 'city', 'state_code', 'country_code', 'latitude',
       'longitude'],
      dtype='object')

In [12]:
#voy a seleccionar las category_code para buscar las empresas desing
df.category_code.unique()

array(['enterprise', 'software', 'network_hosting', 'web', 'games_video',
       'news', 'mobile', 'social', 'music', 'search', 'advertising',
       'messaging', 'photo_video', 'security', 'finance', 'ecommerce',
       'travel', 'hardware', 'public_relations', 'other', 'real_estate',
       'semiconductor', 'analytics', 'health', 'legal', 'sports',
       'biotech', 'cleantech', 'education', 'consulting',
       'transportation', 'hospitality', 'fashion', 'nonprofit',
       'nanotech', 'automotive', None, 'design', 'manufacturing',
       'government', 'local', 'medical'], dtype=object)

In [13]:
#borro las null
drop_null_category = list(df[(df.category_code.isnull() == True)].index)
df = df.drop(drop_null_category)
df.head(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,NaN,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,NaN,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,None,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,NaN,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
4,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,NaN,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431


In [14]:
#compruebo que se han borrado las null
df.category_code.unique()

array(['enterprise', 'software', 'network_hosting', 'web', 'games_video',
       'news', 'mobile', 'social', 'music', 'search', 'advertising',
       'messaging', 'photo_video', 'security', 'finance', 'ecommerce',
       'travel', 'hardware', 'public_relations', 'other', 'real_estate',
       'semiconductor', 'analytics', 'health', 'legal', 'sports',
       'biotech', 'cleantech', 'education', 'consulting',
       'transportation', 'hospitality', 'fashion', 'nonprofit',
       'nanotech', 'automotive', 'design', 'manufacturing', 'government',
       'local', 'medical'], dtype=object)

In [15]:
#me quedo solo con las empresas design
design = df[df.category_code.isin(["design"])]
design.head(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
2028,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
2029,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,Australia,204 Wellington St.,,3065,Collingwood,None,AUS,-37.802659,144.986855
2030,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,Europe,Schlesische Str. 29-30,,,Berlin,None,DEU,52.498620,13.446903
13923,52cdef7e4bab8bd67529ba4e,Graticle,graticle,http://www.crunchbase.com/company/graticle,http://www.graticle.com,http://www.graticle.com/blog/,,graticle,design,1.0,...,None,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144
14523,52cdef7e4bab8bd67529bd1a,Kickstarter,kickstarter,http://www.crunchbase.com/company/kickstarter,http://www.kickstarter.com,http://www.kickstarter.com/blog,,kickstarter,design,NaN,...,None,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN


In [16]:
# Borro las empresas que no tienen coordenadas
null= list(design[((design.latitude.isnull() == True) | (design.longitude.isnull() == True))].index)
design = design.drop(null)
design.head(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
2028,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
2029,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,Australia,204 Wellington St.,,3065,Collingwood,None,AUS,-37.802659,144.986855
2030,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,Europe,Schlesische Str. 29-30,,,Berlin,None,DEU,52.498620,13.446903
13923,52cdef7e4bab8bd67529ba4e,Graticle,graticle,http://www.crunchbase.com/company/graticle,http://www.graticle.com,http://www.graticle.com/blog/,,graticle,design,1.0,...,None,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144


In [17]:
#en que paises hay empresas design
design.country_code.unique()

array(['USA', 'AUS', 'DEU'], dtype=object)

In [18]:
#me quedo con USA... no queda otro
design = design[design.country_code.isin(["USA"])]
design.head(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
2028,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
13923,52cdef7e4bab8bd67529ba4e,Graticle,graticle,http://www.crunchbase.com/company/graticle,http://www.graticle.com,http://www.graticle.com/blog/,,graticle,design,1.0,...,None,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144


In [19]:
design = design[design.city.isin(["San Francisco"])]
design.head(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
2028,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598


Me quedo con San Francisco, en USA, ya que en Ellensburg, aunque tiene aeropuero cerca,
no tiene mas empresas alrededor

Voy a buscar empresas en San Francisco que hayan ganado 1M$

In [20]:
city = df[df.city.isin(["San Francisco"])] 
city.head(5)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
8,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
13,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,None,,,,,San Francisco,CA,USA,37.775196,-122.419204
19,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,None,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
23,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,"{'valuation_amount': 18100000000, 'valuation_c...",,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924
24,52cdef7c4bab8bd675297da4,Powerset,powerset,http://www.crunchbase.com/company/powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60.0,...,None,None,475 Brannan St,,94107,San Francisco,CA,USA,37.778613,-122.395289


In [21]:
city.total_money_raised.unique()

array(['$45M', '$18.5M', '$25.8M', '$1.16B', '$22.5M', '$32.1M', '$23.4M',
       '$73M', '$4.7M', '$120M', '$60.1M', '$0', '$9M', '$3.1M', '$32.2M',
       '$90M', '$50k', '$34.7M', '$32M', '$4.1M', '$2M', '$15M', '$78M',
       '$16M', '$1M', '$7.5M', '$70M', '$40.4M', '$140M', '$14.3M',
       '$8.5M', '$11M', '$7.18M', '$40.1M', '$52M', '$555M', '$28.7M',
       '$12M', '$7.3M', '€11.5M', '$44.7M', '$100k', '$24.8M', '$43.5M',
       '$1.31M', '$65M', '$50M', '$36.1M', '$89M', '$257M', '$25M',
       '$26M', '$670k', '$4.5M', '$19M', '$40.8M', '$6M', '$8M', '$2.25M',
       '$48.9M', '$22.9M', '$17.6M', '$255M', '$48M', '$248M', '$4.8M',
       '$1.65M', '$17.4M', '$13.1M', '$250k', '$400k', '$4.9M', '$30M',
       '$403M', '$41.8M', '$4M', '$46.3M', '$2.5M', '$3M', '$650k',
       '$21.6M', '$109M', '$24M', '$30.6M', '$58.5M', '$200k', '$12.3M',
       '$10.5M', '$33M', '$8.15M', '$46M', '$900k', '$5.1M', '$61.1M',
       '$1.2M', '$40M', '$10.2M', '$715k', '$2.75M', '$37.7M', '$4

In [23]:
#para cambiar la columna a float y poder operar con ella
def change(value):
    cambio = {'€' : 1.09}
    value_n = {'B' : 10**9, 'M' : 10**6 , 'K' : 10**3}
    moneda = value[0]
    n = value[-1].upper()
    pp = re.findall("[0-9\.]{1,}",value)[0]
    pp = float(pp)
    return pp*value_n.get(n,1)*cambio.get(moneda,1)

In [24]:
city.total_money_raised = city.total_money_raised.apply(change)
city.head()

/home/jose/.local/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
8,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
13,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,None,,,,,San Francisco,CA,USA,37.775196,-122.419204
19,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,None,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
23,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,"{'valuation_amount': 18100000000, 'valuation_c...",,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924
24,52cdef7c4bab8bd675297da4,Powerset,powerset,http://www.crunchbase.com/company/powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60.0,...,None,None,475 Brannan St,,94107,San Francisco,CA,USA,37.778613,-122.395289


In [25]:
#Aqui consigo las compañias con mas de 1M
money = city[(city.total_money_raised > 1000000)]
money.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
8,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,None,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
13,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,None,,,,,San Francisco,CA,USA,37.775196,-122.419204
19,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,None,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
23,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,"{'valuation_amount': 18100000000, 'valuation_c...",,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924
24,52cdef7c4bab8bd675297da4,Powerset,powerset,http://www.crunchbase.com/company/powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60.0,...,None,None,475 Brannan St,,94107,San Francisco,CA,USA,37.778613,-122.395289


In [26]:
#compañias de menos de 10 años
money.founded_year.unique()


array([2004., 2002., 2007., 2006., 2005., 2003., 2008., 1990.,   nan,
       1998., 1999., 2011., 2000., 2001., 2012., 1995., 1996., 1997.,
       2009., 2010., 1994., 1987., 1992.])

In [27]:
#borro los nan
drop_null_category = list(money[(money.founded_year.isnull() == True)].index)
money = money.drop(drop_null_category)
money.founded_year.unique()

array([2004., 2002., 2007., 2006., 2005., 2003., 2008., 1990., 1998.,
       1999., 2011., 2000., 2001., 2012., 1995., 1996., 1997., 2009.,
       2010., 1994., 1987., 1992.])

In [28]:
#paso la columna a int
money["founded_year"]=money["founded_year"].astype(int)

In [29]:
money = money[(city.founded_year >= 2010)]
money.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
676,52cdef7c4bab8bd675297f78,CircleUp,circleup,http://www.crunchbase.com/company/circleup,http://circleup.com,https://circleup.com/blog/,,CircleUp,finance,11.0,...,None,,,,,San Francisco,CA,USA,NaN,NaN
1383,52cdef7c4bab8bd6752981d8,Widgetbox,widgetbox,http://www.crunchbase.com/company/widgetbox,http://widgetbox.com,,http://blog.widgetbox.com/feed/,widgetbox,other,30.0,...,None,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599
6149,52cdef7d4bab8bd675299583,Integrate,integrate,http://www.crunchbase.com/company/integrate,http://integrate.com,http://integrate.com/blog,http://blog.integrate.com/?feed=rss2,Integrate,software,120.0,...,None,San Francisco Office,23 Geary St,10th Floor,94104,San Francisco,CA,USA,NaN,NaN
6902,52cdef7d4bab8bd67529990d,Integrate,integrate,http://www.crunchbase.com/company/integrate,http://integrate.com,http://integrate.com/blog,http://blog.integrate.com/?feed=rss2,Integrate,software,120.0,...,None,San Francisco Office,23 Geary St,10th Floor,94104,San Francisco,CA,USA,NaN,NaN
8299,52cdef7d4bab8bd675299fd5,Webtab,webtab,http://www.crunchbase.com/company/webtab,http://www.webtab.com,http://blog.webtab.com,,BartabInc,mobile,NaN,...,None,Corporate Headquarters,847 Sansome Street,Suite L,94111,San Francisco,CA,USA,NaN,NaN


In [30]:
# Borro las empresas que no tienen coordenadas
null= list(money[((money.latitude.isnull() == True) | (money.longitude.isnull() == True))].index)
money = money.drop(null)
money

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
1383,52cdef7c4bab8bd6752981d8,Widgetbox,widgetbox,http://www.crunchbase.com/company/widgetbox,http://widgetbox.com,,http://blog.widgetbox.com/feed/,widgetbox,other,30.0,...,None,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599
12371,52cdef7e4bab8bd67529b2a6,MyLikes,mylikes,http://www.crunchbase.com/company/mylikes,http://mylikes.com,http://blog.mylikes.com,feed://blog.mylikes.com/atom.xml,MyLikes,advertising,NaN,...,None,,"1000 Brannan St, #502",,94103,San Francisco,CA,USA,37.777995,-122.408914
14528,52cdef7e4bab8bd67529bd23,Fliptop,fliptop,http://www.crunchbase.com/company/fliptop,http://www.fliptop.com,http://blog.fliptop.com,https://chrome.google.com/extensions/detail/cp...,fliptop,enterprise,16.0,...,None,Headquarters,580 Howard Street,Suite 201,94105,San Francisco,CA,USA,37.775206,-122.419209


In [31]:
#uno los data money y design

dat = pd.concat([money, design], axis=0)
dat

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,ipo,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
1383,52cdef7c4bab8bd6752981d8,Widgetbox,widgetbox,http://www.crunchbase.com/company/widgetbox,http://widgetbox.com,,http://blog.widgetbox.com/feed/,widgetbox,other,30.0,...,None,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599
12371,52cdef7e4bab8bd67529b2a6,MyLikes,mylikes,http://www.crunchbase.com/company/mylikes,http://mylikes.com,http://blog.mylikes.com,feed://blog.mylikes.com/atom.xml,MyLikes,advertising,NaN,...,None,,"1000 Brannan St, #502",,94103,San Francisco,CA,USA,37.777995,-122.408914
14528,52cdef7e4bab8bd67529bd23,Fliptop,fliptop,http://www.crunchbase.com/company/fliptop,http://www.fliptop.com,http://blog.fliptop.com,https://chrome.google.com/extensions/detail/cp...,fliptop,enterprise,16.0,...,None,Headquarters,580 Howard Street,Suite 201,94105,San Francisco,CA,USA,37.775206,-122.419209
2028,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,None,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598


Paso dat a geopandas

In [32]:

gdf = gpd.GeoDataFrame(dat, geometry=gpd.points_from_xy(dat.longitude, dat.latitude))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geometry
1383,52cdef7c4bab8bd6752981d8,Widgetbox,widgetbox,http://www.crunchbase.com/company/widgetbox,http://widgetbox.com,,http://blog.widgetbox.com/feed/,widgetbox,other,30.0,...,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599,POINT (-122.39860 37.79885)
12371,52cdef7e4bab8bd67529b2a6,MyLikes,mylikes,http://www.crunchbase.com/company/mylikes,http://mylikes.com,http://blog.mylikes.com,feed://blog.mylikes.com/atom.xml,MyLikes,advertising,NaN,...,,"1000 Brannan St, #502",,94103,San Francisco,CA,USA,37.777995,-122.408914,POINT (-122.40891 37.77799)
14528,52cdef7e4bab8bd67529bd23,Fliptop,fliptop,http://www.crunchbase.com/company/fliptop,http://www.fliptop.com,http://blog.fliptop.com,https://chrome.google.com/extensions/detail/cp...,fliptop,enterprise,16.0,...,Headquarters,580 Howard Street,Suite 201,94105,San Francisco,CA,USA,37.775206,-122.419209,POINT (-122.41921 37.77521)
2028,52cdef7c4bab8bd675298447,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,...,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598,POINT (-122.40060 37.79553)


In [33]:
#crs={'init':'epsg:4236'}
#gdf=GeoDataFrame(gdf, crs=crs)
#gdf
gdf.drop(columns='_id', inplace=True)
gdf.head()


,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,...,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geometry
1383,Widgetbox,widgetbox,http://www.crunchbase.com/company/widgetbox,http://widgetbox.com,,http://blog.widgetbox.com/feed/,widgetbox,other,30.0,2012.0,...,None,None,None,94111,San Francisco,CA,USA,37.798853,-122.398599,POINT (-122.39860 37.79885)
12371,MyLikes,mylikes,http://www.crunchbase.com/company/mylikes,http://mylikes.com,http://blog.mylikes.com,feed://blog.mylikes.com/atom.xml,MyLikes,advertising,NaN,2010.0,...,,"1000 Brannan St, #502",,94103,San Francisco,CA,USA,37.777995,-122.408914,POINT (-122.40891 37.77799)
14528,Fliptop,fliptop,http://www.crunchbase.com/company/fliptop,http://www.fliptop.com,http://blog.fliptop.com,https://chrome.google.com/extensions/detail/cp...,fliptop,enterprise,16.0,2010.0,...,Headquarters,580 Howard Street,Suite 201,94105,San Francisco,CA,USA,37.775206,-122.419209,POINT (-122.41921 37.77521)
2028,99designs,99designs,http://www.crunchbase.com/company/99designs,http://99designs.com,http://99designs.com/designer-blog/,http://99designs.com/designer-blog/feed/,99designs,design,100.0,2008.0,...,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598,POINT (-122.40060 37.79553)


Empiezo con la API de Google

In [37]:
#conocer las coordenadas de un sitio
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

In [38]:
SF = geocode("San Francisco")
SF

{'coordinates': [-15.41914, 27.99493], 'type': 'Point'}

In [90]:
#hace las peticiones a la API de google
def requestMaps(radius,keyword,long,lat,tipo):
    token = os.getenv("API_KEY")
    if not token:
        raise ValueError("You must set a GITHUB_APIKEY token")
    baseUrl = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={token}&location={lat},{long}&radius={radius}&type={tipo}&keyword={keyword}"
    url = baseUrl
    print(f"Requesting data from {url[:6]}")
    res = requests.get(url)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

Restaurantes Veganos:

In [41]:
radius=2000
keyword='vegano'
tipo='restaurant'
long=-122.400598
lat=37.795531
vegan = requestMaps(radius,keyword,long,lat,tipo)
vegan

Requesting data from https:


{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 37.7813183,
     'lng': -122.4166966},
    'viewport': {'northeast': {'lat': 37.78276482989272,
      'lng': -122.4153731201073},
     'southwest': {'lat': 37.78006517010728, 'lng': -122.4180727798928}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
   'id': 'c71bef699e4eeaa286b43320bd66f91bbafec6ac',
   'name': 'Golden Era',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103322296802546171058">Veronica Lin</a>'],
     'photo_reference': 'CmRaAAAA6A4_jfN9TIxExHYkQQH3gSs0zwVNx_LHxdg_nMZ4s4uN3Hky79NVLLQ9dZec9uB7TckbQVD0V7xfiffNUNo6L8H4QiAsphN3UEoWhjS_IY2hdx-3WrQzQMQNkPpAQ8nMEhBa-kSDGqdXmxT3EJb3IgefGhSatgB3uq-9zKXTmuf1zjJ7Qrs00Q',
     'width': 4032}],
   'place_id': 'ChIJ73guiQ73MhURXUofBQbpcAo',
   'plus_code': {'compound_code': 'QHJM+G8 Tenderloin, San Francisco, CA',
    'global_

In [42]:
veg=pd.json_normalize(vegan["results"])
veg.head()

,icon,id,name,photos,place_id,price_level,rating,reference,scope,types,...,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code
0,https://maps.gstatic.com/mapfiles/place_api/ic...,c71bef699e4eeaa286b43320bd66f91bbafec6ac,Golden Era,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ73guiQ73MhURXUofBQbpcAo,1.0,4.6,ChIJ73guiQ73MhURXUofBQbpcAo,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"395 Golden Gate Ave, San Francisco",37.781318,-122.416697,37.782765,-122.415373,37.780065,-122.418073,False,"QHJM+G8 Tenderloin, San Francisco, CA",849VQHJM+G8
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6f245b54e463f8096af36862e912c5cd3b4ac45a,Loving Hut,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJy4kpbIaAhYARmklXpiuj-pE,1.0,4.4,ChIJy4kpbIaAhYARmklXpiuj-pE,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"865 Market St, San Francisco",37.784307,-122.406012,37.785616,-122.405092,37.782917,-122.408577,False,"QHMV+PH SoMa, San Francisco, CA",849VQHMV+PH
2,https://maps.gstatic.com/mapfiles/place_api/ic...,70e15cb79c844fbf64b5d0d5295e6a029e8f5778,The Lucky Creation Vegetarian Restaurant,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJQdwWVPOAhYAR6ZL-cDxRC5s,1.0,4.2,ChIJQdwWVPOAhYAR6ZL-cDxRC5s,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"854 Washington St, San Francisco",37.795125,-122.407409,37.796433,-122.406052,37.793734,-122.408752,False,"QHWV+22 Chinatown, San Francisco, CA",849VQHWV+22
3,https://maps.gstatic.com/mapfiles/place_api/ic...,f6cb4b4d4137713891969cbb3a7813e9cf40f21a,Ananda Fuara,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJiaYceJyAhYARjyA70dD--UA,1.0,4.5,ChIJiaYceJyAhYARjyA70dD--UA,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"1298 Market St, San Francisco",37.777875,-122.416294,37.779167,-122.414977,37.776467,-122.417677,False,"QHHM+5F Civic Center, San Francisco, CA",849VQHHM+5F
4,https://maps.gstatic.com/mapfiles/place_api/ic...,04e14977bd7f5159aaad40863fd507b18382885b,Nourish Cafe,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ19cYa5OAhYARvyZWBkZiZIk,2.0,4.4,ChIJ19cYa5OAhYARvyZWBkZiZIk,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"1030 Hyde St, San Francisco",37.790518,-122.417079,37.791869,-122.415848,37.789169,-122.418548,False,"QHRM+65 Nob Hill, San Francisco, CA",849VQHRM+65


Starbucks

In [81]:
radius=2000
keyword='starbucks'
tipo='cafeteria'
long=-122.400598
lat=37.795531
star = requestMaps(radius,keyword,long,lat,tipo)
star

Requesting data from https:


{'html_attributions': [],
 'next_page_token': 'CrQCKwEAAEkDJ5KMYvXaZlmlYshTzMlSWjY9jDBFxErAr2qTNwCXhq4vV8IN98CKJyfIg3THrYYZbns7DAUxUTUAd1N9mGbnhbPs_GC5Cw-LigTR2eQiE7HBqlC27SI0Y6LJXL3FXqyWERh0-dhe-vBc7kZ1OCiRvbEVQNA6uegAXqtWNn5stvaEZRj0kWoU4KZ_JAduIjELvbzf8RDS-QWw6JqfIVytE6CCbBuZQ7G3rDvV2zBSmefZ4kQxTnXPmNuxaR94xQvWdkBdj2nvd42UCE4-Ur9GW7jnrI0Qutc6fJJXxQBJngvwEL3hDrqw4YLbeLN1FroNolg4vnzypJwT5WAXvUKYcqclUlYKHcYBUYdj5kHeavYuyEknhyhEQOP0kunkTKXAmcBHLt9VIdOgUZw4BiMSEK80Xb-Rsn-FI3NrVPWoJckaFPcw9pAm5b37NdXpdzPzRxhfZ-OD',
 'results': [{'geometry': {'location': {'lat': 37.789795, 'lng': -122.396943},
    'viewport': {'northeast': {'lat': 37.79129982989272,
      'lng': -122.3957026201072},
     'southwest': {'lat': 37.78860017010728, 'lng': -122.3984022798927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png',
   'id': 'a20d2c2f08be1c2311385a454837cc7840d9fd01',
   'name': 'Starbucks',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 3024,
     'html_a

In [44]:
star=pd.json_normalize(star["results"])
star.head()

,icon,id,name,photos,place_id,price_level,rating,reference,scope,types,...,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code
0,https://maps.gstatic.com/mapfiles/place_api/ic...,a20d2c2f08be1c2311385a454837cc7840d9fd01,Starbucks,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJx7z69f-BhYARZBTkZC_Px3E,2,4.8,ChIJx7z69f-BhYARZBTkZC_Px3E,GOOGLE,"[cafe, food, point_of_interest, store, establi...",...,"101 1st St Floor 5, San Francisco",37.789795,-122.396943,37.791300,-122.395703,37.788600,-122.398402,False,"QJQ3+W6 SoMa, San Francisco, CA, United States",849VQJQ3+W6
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6b6c8a393f987b4ca9e5509f002fa676502c0408,Starbucks,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJSdu8souAhYARm2j_V4uodVM,2,4.1,ChIJSdu8souAhYARm2j_V4uodVM,GOOGLE,"[cafe, food, point_of_interest, store, establi...",...,"580 California Street, California St Suite 150...",37.792814,-122.404268,37.794100,-122.402899,37.791401,-122.405599,False,"QHVW+47 Financial District, San Francisco, CA,...",849VQHVW+47
2,https://maps.gstatic.com/mapfiles/place_api/ic...,ca6f7503c0281c46ed709880acae0b4d2dc432e5,Starbucks,"[{'height': 1951, 'html_attributions': ['<a hr...",ChIJdWhPEOGAhYARHNvv_2M7DgI,2,4.2,ChIJdWhPEOGAhYARHNvv_2M7DgI,GOOGLE,"[cafe, food, point_of_interest, store, establi...",...,"685 Beach St, San Francisco",37.806576,-122.420411,37.807973,-122.419103,37.805273,-122.421802,False,"RH4H+JR Fisherman's Wharf, San Francisco, CA, ...",849VRH4H+JR
3,https://maps.gstatic.com/mapfiles/place_api/ic...,b7e49507cff97519826981411c6ed345c03859a3,Starbucks,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJrZxI34WAhYARLlHHWVCHXa4,2,4.1,ChIJrZxI34WAhYARLlHHWVCHXa4,GOOGLE,"[cafe, food, point_of_interest, store, establi...",...,"865 Market Street C, #26A, San Francisco",37.784078,-122.407668,37.785573,-122.406225,37.782873,-122.408925,False,"QHMR+JW SoMa, San Francisco, CA, United States",849VQHMR+JW
4,https://maps.gstatic.com/mapfiles/place_api/ic...,c7ff29596052ec6ddfc18a9e6d8120e0fdbaa698,Starbucks,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJO0psvY6AhYARW4wjPcTZcdM,2,4.0,ChIJO0psvY6AhYARW4wjPcTZcdM,GOOGLE,"[cafe, food, point_of_interest, store, establi...",...,"462 Powell St, San Francisco",37.789146,-122.408456,37.790487,-122.407177,37.787787,-122.409876,False,"QHQR+MJ Lower Nob Hill, San Francisco, CA, Uni...",849VQHQR+MJ


basketball court

In [45]:
radius=5000
keyword='basketball'
long=-122.400598
tipo='basketball courts'
lat=37.795531
bask = requestMaps(radius,keyword,long,lat,tipo)
bask

Requesting data from https:


{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 37.7748488,
     'lng': -122.4218618},
    'viewport': {'northeast': {'lat': 37.77634627989272,
      'lng': -122.4204147201072},
     'southwest': {'lat': 37.77364662010728, 'lng': -122.4231143798927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
   'id': '65dd55d4ed0559fb804a03a06998a16f4bce2d21',
   'name': 'Hooked On Hoops | San Francisco Basketball and Athlete Development Academy',
   'opening_hours': {'open_now': False},
   'place_id': 'ChIJh8w4I5-AhYAR9l7ZMq_0tEU',
   'plus_code': {'compound_code': 'QHFH+W7 Hayes Valley, San Francisco, CA, United States',
    'global_code': '849VQHFH+W7'},
   'rating': 0,
   'reference': 'ChIJh8w4I5-AhYAR9l7ZMq_0tEU',
   'scope': 'GOOGLE',
   'types': ['school', 'point_of_interest', 'establishment'],
   'user_ratings_total': 0,
   'vicinity': '151 Oak St, San Francisco'},
  {'geometry': {'location': {'lat': 37.778052, 'lng': -122

In [46]:
bask=pd.json_normalize(bask["results"])
bask.head()

,icon,id,name,place_id,rating,reference,scope,types,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,photos
0,https://maps.gstatic.com/mapfiles/place_api/ic...,65dd55d4ed0559fb804a03a06998a16f4bce2d21,Hooked On Hoops | San Francisco Basketball and...,ChIJh8w4I5-AhYAR9l7ZMq_0tEU,0.0,ChIJh8w4I5-AhYAR9l7ZMq_0tEU,GOOGLE,"[school, point_of_interest, establishment]",0,"151 Oak St, San Francisco",37.774849,-122.421862,37.776346,-122.420415,37.773647,-122.423114,False,"QHFH+W7 Hayes Valley, San Francisco, CA, Unite...",849VQHFH+W7,NaN
1,https://maps.gstatic.com/mapfiles/place_api/ic...,4ca428cfc56129405b8dfbfb5733d298e32e3293,Midnight Basketball League Programs,ChIJ42QDIqOAhYARicPDmf2lNs0,5.0,ChIJ42QDIqOAhYARicPDmf2lNs0,GOOGLE,"[point_of_interest, establishment]",1,"635 Fulton St, San Francisco",37.778052,-122.427279,37.779502,-122.425950,37.776803,-122.428650,NaN,"QHHF+63 Western Addition, San Francisco, CA, U...",849VQHHF+63,NaN
2,https://maps.gstatic.com/mapfiles/place_api/ic...,9191483659b514edde3d88f6f3a6f602a99113f4,elite basketball,ChIJq6p6_sR_j4ARdNdTzm84KGw,0.0,ChIJq6p6_sR_j4ARdNdTzm84KGw,GOOGLE,"[point_of_interest, establishment]",0,"500 Terry A Francois Blvd, San Francisco",37.770372,-122.387125,37.771709,-122.386087,37.769009,-122.388787,NaN,"QJC7+45 SoMa, San Francisco, CA, United States",849VQJC7+45,NaN
3,https://maps.gstatic.com/mapfiles/place_api/ic...,131c08b63de1338b31e89b4848e3239415a2f08e,Mission Creek Park-Basketball Court,ChIJ98fdU9J_j4ARDYgb-gwM6tE,4.2,ChIJ98fdU9J_j4ARDYgb-gwM6tE,GOOGLE,"[point_of_interest, establishment]",26,"401 Berry St, San Francisco",37.772088,-122.397716,37.773504,-122.396449,37.770804,-122.399148,NaN,"QJC2+RW SoMa, San Francisco, CA, United States",849VQJC2+RW,"[{'height': 3024, 'html_attributions': ['<a hr..."
4,https://maps.gstatic.com/mapfiles/place_api/ic...,fb0ab60c21d3c9c87c95c7cca26583ea2e9f09fe,Mission Dolores Park basketball court,ChIJEWTJxxl-j4ARXJJ24nCiZjI,4.4,ChIJEWTJxxl-j4ARXJJ24nCiZjI,GOOGLE,"[point_of_interest, establishment]",8,San Francisco,37.761090,-122.427687,37.762440,-122.426337,37.759741,-122.429037,NaN,"QH6C+CW Mission District, San Francisco, CA, U...",849VQH6C+CW,NaN


Guarderias

In [47]:
radius=2000
keyword='guarderia'
long=-122.400598
tipo='guarderia'
lat=37.795531
guar = requestMaps(radius,keyword,long,lat,tipo)
guar

Requesting data from https:


{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 37.803419, 'lng': -122.4151504},
    'viewport': {'northeast': {'lat': 37.80482792989272,
      'lng': -122.4138129701072},
     'southwest': {'lat': 37.80212827010727, 'lng': -122.4165126298927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png',
   'id': '9f0410eeddda0dba4b0a2f789a72a03d8365fdf7',
   'name': "Chinatown Community Children's Center",
   'opening_hours': {'open_now': False},
   'place_id': 'ChIJaySS01aBhYARlEYonkDqo6s',
   'plus_code': {'compound_code': 'RH3M+9W Russian Hill, San Francisco, CA, United States',
    'global_code': '849VRH3M+9W'},
   'rating': 0,
   'reference': 'ChIJaySS01aBhYARlEYonkDqo6s',
   'scope': 'GOOGLE',
   'types': ['point_of_interest', 'establishment'],
   'user_ratings_total': 0,
   'vicinity': '715 Chestnut St, San Francisco'},
  {'geometry': {'location': {'lat': 37.8007762, 'lng': -122.4202329},
    'viewport': {'northeast': {'lat': 37.80222

In [48]:
guar=pd.json_normalize(guar["results"])
guar.head()

,icon,id,name,place_id,rating,reference,scope,types,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,photos
0,https://maps.gstatic.com/mapfiles/place_api/ic...,9f0410eeddda0dba4b0a2f789a72a03d8365fdf7,Chinatown Community Children's Center,ChIJaySS01aBhYARlEYonkDqo6s,0,ChIJaySS01aBhYARlEYonkDqo6s,GOOGLE,"[point_of_interest, establishment]",0,"715 Chestnut St, San Francisco",37.803419,-122.415150,37.804828,-122.413813,37.802128,-122.416513,False,"RH3M+9W Russian Hill, San Francisco, CA, Unite...",849VRH3M+9W,NaN
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6db172594d364c563210a821ec9ef631489a7dbe,Riley and Tyler's Pad,ChIJlwXNYRaBhYARcxs_6E6nDXc,0,ChIJlwXNYRaBhYARcxs_6E6nDXc,GOOGLE,"[point_of_interest, establishment]",0,"1221 Greenwich St Apt 8, San Francisco",37.800776,-122.420233,37.802224,-122.418903,37.799524,-122.421603,NaN,"RH2H+8W Russian Hill, San Francisco, CA, Unite...",849VRH2H+8W,NaN
2,https://maps.gstatic.com/mapfiles/place_api/ic...,bca8199cf4f95cc9daf5ebdeae93a56972de7ce7,C5 Children’s School - Under 2,ChIJV20AQ9CBhYARtpoojTazrak,0,ChIJV20AQ9CBhYARtpoojTazrak,GOOGLE,"[point_of_interest, establishment]",0,"525 Golden Gate Ave, San Francisco",37.780858,-122.418947,37.782392,-122.417642,37.779693,-122.420342,NaN,"QHJJ+8C Tenderloin, San Francisco, CA, United ...",849VQHJJ+8C,NaN
3,https://maps.gstatic.com/mapfiles/place_api/ic...,2f9fcbd2000df5b97532c4b9c7a3b263459d8ad6,Marin Day Schools Fremont Street,ChIJ--okrmSAhYAR4qIfL2LIC2I,0,ChIJ--okrmSAhYAR4qIfL2LIC2I,GOOGLE,"[school, point_of_interest, establishment]",0,"342 Howard St, San Francisco",37.789506,-122.395069,37.790908,-122.393587,37.788209,-122.396286,False,"QJQ3+RX SoMa, San Francisco, CA, United States",849VQJQ3+RX,"[{'height': 1266, 'html_attributions': ['<a hr..."
4,https://maps.gstatic.com/mapfiles/place_api/ic...,a133228012dba2b0766ea82eeb69188706c6c4c3,Pacific Gas Children's Center,ChIJdaK7j2OAhYARx9WiGHeSjCg,2,ChIJdaK7j2OAhYARx9WiGHeSjCg,GOOGLE,"[point_of_interest, establishment]",1,"77 Beale St, San Francisco",37.792128,-122.396679,37.793303,-122.395187,37.790603,-122.397886,NaN,"QJR3+V8 SoMa, San Francisco, CA, United States",849VQJR3+V8,NaN


Bar

In [49]:
radius=2000
keyword='nightclub'
long=-122.400598
tipo='nightclub'
lat=37.795531
bar = requestMaps(radius,keyword,long,lat,tipo)
bar

Requesting data from https:


{'html_attributions': [],
 'next_page_token': 'CrQCKwEAAH6aVCR-DIQs3zyaTnDIUSqXBVkZinGhDLzpG2BNEGPVPCtdHZS0n2dC8_w6A9euzJcTrQVi_OqBSkBEn3UsYyAoMOU4lX3B35ppPsUaOL7fE5r9mho_YixCeTQzdz-TKl_f-NoI7qFdS4ItipB50Gk4EYUctNUwZJd9h7i17o1quZPOyDHpYdQEIBFsRjtK7XutmeO-ZkcduYOgvqsoDroj-8J6zHcelxzhiaXPMhyXIgAzqO8Wc70PcgxMAVsXO7qPCP_FwrBuaRiuAiHtIf25pHa6Pj0uPcnG9151O-QOQhHoRkOlMKhYo16Id865-lDJ5Rne0sq9cPupgaD7nM_aF6IhDdhkfNhpt9xBW_ed6cPAb1328tOFDspgY7IvpqZ-79Lg_POAzLA9j085gDsSECZ7qeFVuVYxdlZ2P189_OcaFKph451MG65Ra81_RvjzY6t6_cFT',
 'results': [{'geometry': {'location': {'lat': 37.787956, 'lng': -122.3972472},
    'viewport': {'northeast': {'lat': 37.78921917989272,
      'lng': -122.3957777701073},
     'southwest': {'lat': 37.78651952010728, 'lng': -122.3984774298927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/bar-71.png',
   'id': 'f531b08a612484481d0ab2ddf84033383eb729d0',
   'name': 'Temple Nightclub San Francisco',
   'opening_hours': {'open_now': False},
   'photos': [{'height

In [50]:
bar=pd.json_normalize(bar["results"])
bar.head()

,icon,id,name,photos,place_id,rating,reference,scope,types,user_ratings_total,...,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,price_level
0,https://maps.gstatic.com/mapfiles/place_api/ic...,f531b08a612484481d0ab2ddf84033383eb729d0,Temple Nightclub San Francisco,"[{'height': 638, 'html_attributions': ['<a hre...",ChIJvwR28w-BhYARQAAkwF3pDHM,3.5,ChIJvwR28w-BhYARQAAkwF3pDHM,GOOGLE,"[night_club, bar, point_of_interest, establish...",427,...,37.787956,-122.397247,37.789219,-122.395778,37.786520,-122.398477,False,"QJQ3+54 SoMa, San Francisco, CA, United States",849VQJQ3+54,NaN
1,https://maps.gstatic.com/mapfiles/place_api/ic...,18e74c4aa201e64722d8414f3166d180244d8aa9,The Grand,"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJO6fCZH-AhYARbmHHgxnFxB8,3.5,ChIJO6fCZH-AhYARbmHHgxnFxB8,GOOGLE,"[night_club, point_of_interest, establishment]",116,...,37.779100,-122.397900,37.780506,-122.396482,37.777807,-122.399181,False,"QJH2+JR SoMa, San Francisco, CA, United States",849VQJH2+JR,3.0
2,https://maps.gstatic.com/mapfiles/place_api/ic...,3b3b1a25037fe8a2aa47a8178b0324480e69628e,Monarch,"[{'height': 635, 'html_attributions': ['<a hre...",ChIJ328vE4SAhYARpJpsQ3UcwHQ,4.1,ChIJ328vE4SAhYARpJpsQ3UcwHQ,GOOGLE,"[night_club, point_of_interest, establishment]",397,...,37.780965,-122.408474,37.782365,-122.407197,37.779665,-122.409897,False,"QHJR+9J SoMa, San Francisco, CA, United States",849VQHJR+9J,2.0
3,https://maps.gstatic.com/mapfiles/place_api/ic...,f7b4baafbb37bf537134c1743bc77e4750702325,HUE,"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJFXzz-PSAhYARdawj6Vezo9g,3.8,ChIJFXzz-PSAhYARdawj6Vezo9g,GOOGLE,"[night_club, bar, point_of_interest, establish...",69,...,37.797949,-122.404718,37.799385,-122.403383,37.796685,-122.406082,False,"QHXW+54 Financial District, San Francisco, CA,...",849VQHXW+54,2.0
4,https://maps.gstatic.com/mapfiles/place_api/ic...,e30254400171df555b96784f93c4168a4cdb511a,1015 Folsom,"[{'height': 1152, 'html_attributions': ['<a hr...",ChIJRwXhl4GAhYARKMdYXvUW1m8,3.8,ChIJRwXhl4GAhYARKMdYXvUW1m8,GOOGLE,"[night_club, bar, point_of_interest, establish...",264,...,37.778112,-122.405800,37.779552,-122.404513,37.776852,-122.407213,False,"QHHV+6M SoMa, San Francisco, CA, United States",849VQHHV+6M,2.0


Aeropuerto

In [69]:
radius=30000
keyword='aeropuerto'
long=-122.400598
tipo='aeropuerto'
lat=37.795531
air = requestMaps(radius,keyword,long,lat,tipo)
air

Requesting data from https:


{'html_attributions': [],
 'next_page_token': 'CrQCLQEAANOjtxYa4FiAxb00YynsUDHNTvz4fCDXvWUh3j3LmxPIhx-xkbtrKpv0cVWzqyaDcO_TA2I-BvMIvo6utCacHvZwFXDFNx5M2i7TKS0tPOnkLt4NFHHsjk4lBCoYb57cLO2mmRRSRxm4A0wjEU4DhStv0H8VeIp7v5cXj6KiMM3x5pAMJToQ50WA2MvXeIG4TRYHi_hsKea56TsSvImGOX9yKaWRwXdN2oBmsH06uBGw2xPUD5P041U9flB1Bw84h8TvYOJTrPE6O2Xie6yZ0aP6t3iBPG7cKf-QtFwJtpKoqLe-QUcC0KDN6199yt6xA66AbYapZYWhLRLKtRsc724qyVhBdfJqx98EPFH-GhsA-iWkQCWoId7uXD9fxMqicWKShNdlAb1tBgXYTLYUX2QSEMUolTmZ3s4lQ17olPNOGYkaFJz4-nRb7ArMu8Jz1hVlvj5Lb-Fh',
 'results': [{'geometry': {'location': {'lat': 37.6213129,
     'lng': -122.3789554},
    'viewport': {'northeast': {'lat': 37.62722864999999, 'lng': -122.36840685},
     'southwest': {'lat': 37.60356565, 'lng': -122.41060105}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/airport-71.png',
   'id': '9c9fb107d7a776c3f6a26791b07982d7db079113',
   'name': 'San Francisco International Airport',
   'photos': [{'height': 2658,
     'html_attributions': ['<a href="ht

In [70]:
air=pd.json_normalize(air["results"])
air.head()

,icon,id,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,plus_code.compound_code,plus_code.global_code,opening_hours.open_now
0,https://maps.gstatic.com/mapfiles/place_api/ic...,9c9fb107d7a776c3f6a26791b07982d7db079113,San Francisco International Airport,"[{'height': 2658, 'html_attributions': ['<a hr...",ChIJVVVVVYx3j4ARP-3NGldc8qQ,3.9,ChIJVVVVVYx3j4ARP-3NGldc8qQ,GOOGLE,"[airport, point_of_interest, establishment]",6830,San Francisco,37.621313,-122.378955,37.627229,-122.368407,37.603566,-122.410601,"JJCC+GC South San Francisco, California, Unite...",849VJJCC+GC,NaN
1,https://maps.gstatic.com/mapfiles/place_api/ic...,b5aa752b1fa3db2e57c8706cf847e67e06c24d3b,Oakland International Airport,"[{'height': 2723, 'html_attributions': ['<a hr...",ChIJQabAAlSEj4ARYHQBAw8MY7A,3.7,ChIJQabAAlSEj4ARYHQBAw8MY7A,GOOGLE,"[airport, point_of_interest, establishment]",1158,"1 Airport Dr, Oakland",37.712569,-122.219743,37.723874,-122.192786,37.708800,-122.228729,"PQ7J+24 Oakland, California, United States",849VPQ7J+24,NaN
2,https://maps.gstatic.com/mapfiles/place_api/ic...,392e919a25376bd67f213fdda0945103021ea75c,Hayward Executive Airport,"[{'height': 2736, 'html_attributions': ['<a hr...",ChIJYXw7hN-Tj4ARJ9ESaTy5MH4,4.0,ChIJYXw7hN-Tj4ARJ9ESaTy5MH4,GOOGLE,"[airport, point_of_interest, establishment]",30,"20301 Skywest Dr, Hayward",37.662852,-122.121630,37.668340,-122.109770,37.654972,-122.125584,"MV7H+48 Hayward, California, United States",849VMV7H+48,NaN
3,https://maps.gstatic.com/mapfiles/place_api/ic...,34b7fb993a1390b7722eadfd44c9898998048e47,Oakland International Airport-Administrative,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJSdTHzAWFj4ARAOiJ5ZS1IvA,3.4,ChIJSdTHzAWFj4ARAOiJ5ZS1IvA,GOOGLE,"[airport, point_of_interest, establishment]",11,"1 Airport Dr, Oakland",37.712554,-122.214324,37.714091,-122.212593,37.711391,-122.215292,"PQ7P+27 Oakland, California, United States",849VPQ7P+27,NaN
4,https://maps.gstatic.com/mapfiles/place_api/ic...,345736a8befdb3252c0f2e647b844522146673fd,19L CL,NaN,ChIJj0yoIcF3j4ARe0XDWICkG_8,5.0,ChIJj0yoIcF3j4ARe0XDWICkG_8,GOOGLE,"[airport, point_of_interest, establishment]",1,San Francisco,37.628650,-122.366396,37.630000,-122.365046,37.627300,-122.367746,"JJHM+FC South San Francisco, California, Unite...",849VJJHM+FC,NaN


Uno las API

In [53]:
api = pd.concat([veg, star, bask, guar, bar, air], axis=0)
api.head()

,icon,id,name,photos,place_id,price_level,rating,reference,scope,types,...,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code
0,https://maps.gstatic.com/mapfiles/place_api/ic...,c71bef699e4eeaa286b43320bd66f91bbafec6ac,Golden Era,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ73guiQ73MhURXUofBQbpcAo,1.0,4.6,ChIJ73guiQ73MhURXUofBQbpcAo,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"395 Golden Gate Ave, San Francisco",37.781318,-122.416697,37.782765,-122.415373,37.780065,-122.418073,False,"QHJM+G8 Tenderloin, San Francisco, CA",849VQHJM+G8
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6f245b54e463f8096af36862e912c5cd3b4ac45a,Loving Hut,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJy4kpbIaAhYARmklXpiuj-pE,1.0,4.4,ChIJy4kpbIaAhYARmklXpiuj-pE,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"865 Market St, San Francisco",37.784307,-122.406012,37.785616,-122.405092,37.782917,-122.408577,False,"QHMV+PH SoMa, San Francisco, CA",849VQHMV+PH
2,https://maps.gstatic.com/mapfiles/place_api/ic...,70e15cb79c844fbf64b5d0d5295e6a029e8f5778,The Lucky Creation Vegetarian Restaurant,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJQdwWVPOAhYAR6ZL-cDxRC5s,1.0,4.2,ChIJQdwWVPOAhYAR6ZL-cDxRC5s,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"854 Washington St, San Francisco",37.795125,-122.407409,37.796433,-122.406052,37.793734,-122.408752,False,"QHWV+22 Chinatown, San Francisco, CA",849VQHWV+22
3,https://maps.gstatic.com/mapfiles/place_api/ic...,f6cb4b4d4137713891969cbb3a7813e9cf40f21a,Ananda Fuara,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJiaYceJyAhYARjyA70dD--UA,1.0,4.5,ChIJiaYceJyAhYARjyA70dD--UA,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"1298 Market St, San Francisco",37.777875,-122.416294,37.779167,-122.414977,37.776467,-122.417677,False,"QHHM+5F Civic Center, San Francisco, CA",849VQHHM+5F
4,https://maps.gstatic.com/mapfiles/place_api/ic...,04e14977bd7f5159aaad40863fd507b18382885b,Nourish Cafe,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ19cYa5OAhYARvyZWBkZiZIk,2.0,4.4,ChIJ19cYa5OAhYARvyZWBkZiZIk,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"1030 Hyde St, San Francisco",37.790518,-122.417079,37.791869,-122.415848,37.789169,-122.418548,False,"QHRM+65 Nob Hill, San Francisco, CA",849VQHRM+65


In [54]:
api = api.rename(columns={'geometry.location.lat':'latitude','geometry.location.lng':'longitude'})
api

,icon,id,name,photos,place_id,price_level,rating,reference,scope,types,...,vicinity,latitude,longitude,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code
0,https://maps.gstatic.com/mapfiles/place_api/ic...,c71bef699e4eeaa286b43320bd66f91bbafec6ac,Golden Era,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ73guiQ73MhURXUofBQbpcAo,1.0,4.6,ChIJ73guiQ73MhURXUofBQbpcAo,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"395 Golden Gate Ave, San Francisco",37.781318,-122.416697,37.782765,-122.415373,37.780065,-122.418073,False,"QHJM+G8 Tenderloin, San Francisco, CA",849VQHJM+G8
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6f245b54e463f8096af36862e912c5cd3b4ac45a,Loving Hut,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJy4kpbIaAhYARmklXpiuj-pE,1.0,4.4,ChIJy4kpbIaAhYARmklXpiuj-pE,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"865 Market St, San Francisco",37.784307,-122.406012,37.785616,-122.405092,37.782917,-122.408577,False,"QHMV+PH SoMa, San Francisco, CA",849VQHMV+PH
2,https://maps.gstatic.com/mapfiles/place_api/ic...,70e15cb79c844fbf64b5d0d5295e6a029e8f5778,The Lucky Creation Vegetarian Restaurant,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJQdwWVPOAhYAR6ZL-cDxRC5s,1.0,4.2,ChIJQdwWVPOAhYAR6ZL-cDxRC5s,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"854 Washington St, San Francisco",37.795125,-122.407409,37.796433,-122.406052,37.793734,-122.408752,False,"QHWV+22 Chinatown, San Francisco, CA",849VQHWV+22
3,https://maps.gstatic.com/mapfiles/place_api/ic...,f6cb4b4d4137713891969cbb3a7813e9cf40f21a,Ananda Fuara,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJiaYceJyAhYARjyA70dD--UA,1.0,4.5,ChIJiaYceJyAhYARjyA70dD--UA,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"1298 Market St, San Francisco",37.777875,-122.416294,37.779167,-122.414977,37.776467,-122.417677,False,"QHHM+5F Civic Center, San Francisco, CA",849VQHHM+5F
4,https://maps.gstatic.com/mapfiles/place_api/ic...,04e14977bd7f5159aaad40863fd507b18382885b,Nourish Cafe,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ19cYa5OAhYARvyZWBkZiZIk,2.0,4.4,ChIJ19cYa5OAhYARvyZWBkZiZIk,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,"1030 Hyde St, San Francisco",37.790518,-122.417079,37.791869,-122.415848,37.789169,-122.418548,False,"QHRM+65 Nob Hill, San Francisco, CA",849VQHRM+65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,https://maps.gstatic.com/mapfiles/place_api/ic...,1c868b30892d3e916a68269f5e4bbc85e4dcc86c,Signature Flight Support OAK - Oakland Int'l A...,"[{'height': 1512, 'html_attributions': ['<a hr...",ChIJLxKFIq6Fj4AR7H3ZG0A5BFc,NaN,3.5,ChIJLxKFIq6Fj4AR7H3ZG0A5BFc,GOOGLE,"[airport, point_of_interest, establishment]",...,"8433 Earhart Rd, Oakland",37.729994,-122.212760,37.731409,-122.211396,37.728710,-122.214095,True,"PQHP+XV Oakland, California, United States",849VPQHP+XV
16,https://maps.gstatic.com/mapfiles/place_api/ic...,731c7a72c782ebaf4a37e25bba74fa5d5cfa0b13,Signature Flight Support SFO - San Francisco I...,"[{'height': 1263, 'html_attributions': ['<a hr...",ChIJI7lPzCV4j4AR-nlkIV-AMgA,NaN,4.4,ChIJI7lPzCV4j4AR-nlkIV-AMgA,GOOGLE,"[airport, point_of_interest, establishment]",...,"Signature Executive Terminal, 1052 N Access Rd...",37.628330,-122.384845,37.629848,-122.383588,37.627149,-122.386287,True,"JJH8+83 South San Francisco, California, Unite...",849VJJH8+83
17,https://maps.gstatic.com/mapfiles/place_api/ic...,fb2b4be0a7a457924823c100ded9638d42331556,San Rafael Airport,"[{'height': 3760, 'html_attributions': ['<a hr...",ChIJN1xh7j2YhYARo5BpZyIyz1k,NaN,5.0,ChIJN1xh7j2YhYARo5BpZyIyz1k,GOOGLE,"[airport, point_of_interest, establishment]",...,"400 Smith Ranch Rd, San Rafael",38.017187,-122.521548,38.018392,-122.520051,38.015692,-122.522750,NaN,"2F8H+V9 San Rafael,

In [55]:
gapi = gpd.GeoDataFrame(api, geometry=gpd.points_from_xy(api.longitude, api.latitude))
print(f'Tipo: {type(gapi)}')
gapi.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,icon,id,name,photos,place_id,price_level,rating,reference,scope,types,...,latitude,longitude,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,geometry
0,https://maps.gstatic.com/mapfiles/place_api/ic...,c71bef699e4eeaa286b43320bd66f91bbafec6ac,Golden Era,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ73guiQ73MhURXUofBQbpcAo,1.0,4.6,ChIJ73guiQ73MhURXUofBQbpcAo,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.781318,-122.416697,37.782765,-122.415373,37.780065,-122.418073,False,"QHJM+G8 Tenderloin, San Francisco, CA",849VQHJM+G8,POINT (-122.41670 37.78132)
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6f245b54e463f8096af36862e912c5cd3b4ac45a,Loving Hut,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJy4kpbIaAhYARmklXpiuj-pE,1.0,4.4,ChIJy4kpbIaAhYARmklXpiuj-pE,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.784307,-122.406012,37.785616,-122.405092,37.782917,-122.408577,False,"QHMV+PH SoMa, San Francisco, CA",849VQHMV+PH,POINT (-122.40601 37.78431)
2,https://maps.gstatic.com/mapfiles/place_api/ic...,70e15cb79c844fbf64b5d0d5295e6a029e8f5778,The Lucky Creation Vegetarian Restaurant,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJQdwWVPOAhYAR6ZL-cDxRC5s,1.0,4.2,ChIJQdwWVPOAhYAR6ZL-cDxRC5s,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.795125,-122.407409,37.796433,-122.406052,37.793734,-122.408752,False,"QHWV+22 Chinatown, San Francisco, CA",849VQHWV+22,POINT (-122.40741 37.79512)
3,https://maps.gstatic.com/mapfiles/place_api/ic...,f6cb4b4d4137713891969cbb3a7813e9cf40f21a,Ananda Fuara,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJiaYceJyAhYARjyA70dD--UA,1.0,4.5,ChIJiaYceJyAhYARjyA70dD--UA,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.777875,-122.416294,37.779167,-122.414977,37.776467,-122.417677,False,"QHHM+5F Civic Center, San Francisco, CA",849VQHHM+5F,POINT (-122.41629 37.77788)
4,https://maps.gstatic.com/mapfiles/place_api/ic...,04e14977bd7f5159aaad40863fd507b18382885b,Nourish Cafe,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ19cYa5OAhYARvyZWBkZiZIk,2.0,4.4,ChIJ19cYa5OAhYARvyZWBkZiZIk,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.790518,-122.417079,37.791869,-122.415848,37.789169,-122.418548,False,"QHRM+65 Nob Hill, San Francisco, CA",849VQHRM+65,POINT (-122.41708 37.79052)


In [56]:
display(gapi.head())
gapi.shape

,icon,id,name,photos,place_id,price_level,rating,reference,scope,types,...,latitude,longitude,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,geometry
0,https://maps.gstatic.com/mapfiles/place_api/ic...,c71bef699e4eeaa286b43320bd66f91bbafec6ac,Golden Era,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ73guiQ73MhURXUofBQbpcAo,1.0,4.6,ChIJ73guiQ73MhURXUofBQbpcAo,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.781318,-122.416697,37.782765,-122.415373,37.780065,-122.418073,False,"QHJM+G8 Tenderloin, San Francisco, CA",849VQHJM+G8,POINT (-122.41670 37.78132)
1,https://maps.gstatic.com/mapfiles/place_api/ic...,6f245b54e463f8096af36862e912c5cd3b4ac45a,Loving Hut,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJy4kpbIaAhYARmklXpiuj-pE,1.0,4.4,ChIJy4kpbIaAhYARmklXpiuj-pE,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.784307,-122.406012,37.785616,-122.405092,37.782917,-122.408577,False,"QHMV+PH SoMa, San Francisco, CA",849VQHMV+PH,POINT (-122.40601 37.78431)
2,https://maps.gstatic.com/mapfiles/place_api/ic...,70e15cb79c844fbf64b5d0d5295e6a029e8f5778,The Lucky Creation Vegetarian Restaurant,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJQdwWVPOAhYAR6ZL-cDxRC5s,1.0,4.2,ChIJQdwWVPOAhYAR6ZL-cDxRC5s,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.795125,-122.407409,37.796433,-122.406052,37.793734,-122.408752,False,"QHWV+22 Chinatown, San Francisco, CA",849VQHWV+22,POINT (-122.40741 37.79512)
3,https://maps.gstatic.com/mapfiles/place_api/ic...,f6cb4b4d4137713891969cbb3a7813e9cf40f21a,Ananda Fuara,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJiaYceJyAhYARjyA70dD--UA,1.0,4.5,ChIJiaYceJyAhYARjyA70dD--UA,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.777875,-122.416294,37.779167,-122.414977,37.776467,-122.417677,False,"QHHM+5F Civic Center, San Francisco, CA",849VQHHM+5F,POINT (-122.41629 37.77788)
4,https://maps.gstatic.com/mapfiles/place_api/ic...,04e14977bd7f5159aaad40863fd507b18382885b,Nourish Cafe,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ19cYa5OAhYARvyZWBkZiZIk,2.0,4.4,ChIJ19cYa5OAhYARvyZWBkZiZIk,GOOGLE,"[restaurant, food, point_of_interest, establis...",...,37.790518,-122.417079,37.791869,-122.415848,37.789169,-122.418548,False,"QHRM+65 Nob Hill, San Francisco, CA",849VQHRM+65,POINT (-122.41708 37.79052)


(98, 22)

Imprimo las columnas para ver cual puedo juntar

In [57]:
list(gdf)

['name',
 'permalink',
 'crunchbase_url',
 'homepage_url',
 'blog_url',
 'blog_feed_url',
 'twitter_username',
 'category_code',
 'number_of_employees',
 'founded_year',
 'deadpooled_year',
 'tag_list',
 'alias_list',
 'email_address',
 'phone_number',
 'description',
 'created_at',
 'updated_at',
 'overview',
 'image',
 'products',
 'relationships',
 'competitions',
 'providerships',
 'total_money_raised',
 'funding_rounds',
 'investments',
 'acquisition',
 'acquisitions',
 'milestones',
 'video_embeds',
 'screenshots',
 'external_links',
 'partners',
 'founded_month',
 'founded_day',
 'deadpooled_month',
 'deadpooled_day',
 'deadpooled_url',
 'ipo',
 'description',
 'address1',
 'address2',
 'zip_code',
 'city',
 'state_code',
 'country_code',
 'latitude',
 'longitude',
 'geometry']

In [58]:
list(gapi)

['icon',
 'id',
 'name',
 'photos',
 'place_id',
 'price_level',
 'rating',
 'reference',
 'scope',
 'types',
 'user_ratings_total',
 'vicinity',
 'latitude',
 'longitude',
 'geometry.viewport.northeast.lat',
 'geometry.viewport.northeast.lng',
 'geometry.viewport.southwest.lat',
 'geometry.viewport.southwest.lng',
 'opening_hours.open_now',
 'plus_code.compound_code',
 'plus_code.global_code',
 'geometry']

In [59]:
gdf_new = gdf [['name', 'geometry', 'category_code']]
gdf_new

,name,geometry,category_code
1383,Widgetbox,POINT (-122.39860 37.79885),other
12371,MyLikes,POINT (-122.40891 37.77799),advertising
14528,Fliptop,POINT (-122.41921 37.77521),enterprise
2028,99designs,POINT (-122.40060 37.79553),design


In [60]:
gapi.rename(columns={'types': 'category_code'}, inplace=True)
gapi_new = gapi [['name', 'geometry', 'category_code']]
gapi_new

,name,geometry,category_code
0,Golden Era,POINT (-122.41670 37.78132),"[restaurant, food, point_of_interest, establis..."
1,Loving Hut,POINT (-122.40601 37.78431),"[restaurant, food, point_of_interest, establis..."
2,The Lucky Creation Vegetarian Restaurant,POINT (-122.40741 37.79512),"[restaurant, food, point_of_interest, establis..."
3,Ananda Fuara,POINT (-122.41629 37.77788),"[restaurant, food, point_of_interest, establis..."
4,Nourish Cafe,POINT (-122.41708 37.79052),"[restaurant, food, point_of_interest, establis..."
...,...,...,...
15,Signature Flight Support OAK - Oakland Int'l A...,POINT (-122.21276 37.72999),"[airport, point_of_interest, establishment]"
16,Signature Flight Support SFO - San Francisco I...,POINT (-122.38485 37.62833),"[airport, point_of_interest, establishment]"
17,San Rafael Airport,POINT (-122.52155 38.01719),"[airport, point_of_interest, establishment]"
18,App Jet Center-Hayward,POINT (-122.11744 37.65973),"[airport, point_of_interest, establishment]"


In [61]:
Map(Layer(gapi_new))

In [62]:
Map(Layer(gdf_new))


In [63]:
final=pd.merge(gdf_new, gapi_new, on='geometry', how='outer')
final.head(4)

,name_x,geometry,category_code_x,name_y,category_code_y
0,Widgetbox,POINT (-122.39860 37.79885),other,NaN,NaN
1,MyLikes,POINT (-122.40891 37.77799),advertising,NaN,NaN
2,Fliptop,POINT (-122.41921 37.77521),enterprise,NaN,NaN
3,99designs,POINT (-122.40060 37.79553),design,NaN,NaN


In [64]:
Map(Layer(final))